In [1]:
# Autoencoder for Unsupervised Anomaly Detection
# Catherine Beazley 
# 4-18-2019

In [2]:
import pandas as pd
import numpy as np
import keras
from keras.models import Model
from keras.layers import Dense, Input
from keras.datasets import mnist
from keras.regularizers import l1
from keras.optimizers import Adam
import time
from scipy.spatial.distance import euclidean
from copy import deepcopy

Using TensorFlow backend.


In [3]:
data = pd.read_csv('1mill_anomaly_ensemble_df.csv')
data = data[['Time', 'Duration',
       'SrcDevice', 'DstDevice', 'Protocol', 'SrcPort', 'DstPort',
       'SrcPackets', 'DstPackets', 'SrcBytes', 'DstBytes', 'LOF_0.01_Labels',
       'EE_0.01_Lables', 'OCSVM_0.01_Labels']]

In [21]:
if_preds = pd.read_csv('IF_predictions.csv', header=None)

In [22]:
data['IF_0.01_Labels'] = if_preds.values

In [25]:
dataNum = data[['Duration', 'SrcPackets', 'DstPackets', 'SrcBytes', 'DstBytes']]
X = dataNum.values

array([[   0,   11,    7, 1577, 3911],
       [ 815,    6,    6,  474, 2205],
       [   1,    7,    7,  640, 1365],
       ...,
       [   0,    4,    3,  216,  164],
       [   0,    6,    5,  329, 1156],
       [   1,    5,    3,  277, 1562]], dtype=int64)

In [108]:
start = time.time()
input_size = 5
hidden_size = 3
code_size = 3

input_x = Input(shape=(input_size,))
hidden_1 = Dense(hidden_size, activation='relu')(input_x)
code = Dense(code_size, activation='relu')(hidden_1)
hidden_2 = Dense(hidden_size, activation='relu')(code)
output_x = Dense(input_size, activation='linear')(hidden_2)

autoencoder = Model(input_x, output_x)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.fit(X, X, epochs=100, batch_size=50)
end = time.time()
print(end - start)

Epoch 1/100
1000000/1000000 [==============================] - 23s 23us/step - loss: 30291106803696.7070
Epoch 2/100
1000000/1000000 [==============================] - 499s 499us/step - loss: 2945911706181.2681
Epoch 3/100
1000000/1000000 [==============================] - 27s 27us/step - loss: 3851390440707.0298
Epoch 4/100
1000000/1000000 [==============================] - 23s 23us/step - loss: 2570546716922.4370
Epoch 5/100
1000000/1000000 [==============================] - 23s 23us/step - loss: 2576984262346.3335
Epoch 6/100
1000000/1000000 [==============================] - 22s 22us/step - loss: 5414735761087.6289
Epoch 7/100
1000000/1000000 [==============================] - 20s 20us/step - loss: 2592117554518.0332
Epoch 8/100
1000000/1000000 [==============================] - 21s 21us/step - loss: 2445894652153.4585
Epoch 9/100
1000000/1000000 [==============================] - 21s 21us/step - loss: 2588140179240.2749
Epoch 10/100
1000000/1000000 [==============================]

In [109]:
start = time.time()
pred = autoencoder.predict(X)
end = time.time()
print(end - start)

15.207031726837158


In [110]:
pred

array([[   6.6944046,   54.115154 ,   50.599274 , 1588.3701   ,
        3212.344    ],
       [  25.18759  ,   30.831078 ,   28.738445 ,  482.77054  ,
        1821.7576   ],
       [  35.699215 ,   15.131115 ,   11.228365 ,  707.2515   ,
        1047.538    ],
       ...,
       [  49.410664 ,   -3.352008 ,   -7.495219 ,  309.47037  ,
          24.525043 ],
       [  37.899586 ,   12.90485  ,    9.7495575,  387.44107  ,
         878.5071   ],
       [  33.118843 ,   19.96091  ,   17.539236 ,  314.53574  ,
        1231.1807   ]], dtype=float32)

In [37]:
X

array([[   0,   11,    7, 1577, 3911],
       [ 815,    6,    6,  474, 2205],
       [   1,    7,    7,  640, 1365],
       ...,
       [   0,    4,    3,  216,  164],
       [   0,    6,    5,  329, 1156],
       [   1,    5,    3,  277, 1562]], dtype=int64)

In [111]:
dists = np.linalg.norm((pred- X), axis=1)
dists_sorted = deepcopy(dists)

In [112]:
dists_sorted.sort()
cutoff_val = dists_sorted[int(len(data) - 0.01*len(data))]

In [113]:
auto_labels = np.where(dists>=cutoff_val, -1, 1)

In [114]:
len(auto_labels[auto_labels==-1])

10000

In [115]:
data['AE_0.01_Labels'] = auto_labels

In [116]:
data[(data['AE_0.01_Labels']==-1) & (data['IF_0.01_Labels']==-1)]

,Time,Duration,SrcDevice,DstDevice,Protocol,SrcPort,DstPort,SrcPackets,DstPackets,SrcBytes,DstBytes,LOF_0.01_Labels,EE_0.01_Lables,OCSVM_0.01_Labels,IF_0.01_Labels,AE_0.01_Labels
28,227498,17,Comp199879,EnterpriseAppServer,6,Port42755,Port68911,302,576,16049,832485,1,1,1,-1,-1
423,250499,4,Comp417598,Comp501516,6,Port84664,443,605,1046,28399,1557926,1,1,1,-1,-1
815,213419,925,Comp513818,ActiveDirectory,6,Port60708,Port26940,627,611,382127,845256,1,1,1,-1,-1
849,218717,20306,Comp107130,Comp509586,6,Port73981,443,4535,4688,1176024,3582012,1,-1,1,-1,-1
951,197550,125607,Comp107130,Comp801677,6,Port33852,443,38111,13499,22300673,620954,1,-1,1,-1,-1
1159,202442,21255,Comp857436,Comp186884,6,Port97191,443,7017,5985,4581691,279421,1,-1,1,-1,-1
1494,215501,4,EnterpriseAppServer,EnterpriseAppServer,6,Port89716,443,252,1925,59511,2757634,1,1,1,-1,-1
1680,243455,880,EnterpriseAppServer,EnterpriseAppServer,6,Port97451,1433,52124,127876,49503617,184805607,1,-1,1,-1,-1
1822,195496,1020,Comp901712,Comp684843,6,Port50499,443,1461,2658,219224,3349705,1,1,1,-1,-1
1849,203980,137,Comp527729,Comp661433,6,Port70707,80,117622,233179,5414507,349759788,1,-1,1,-1,-1


In [117]:
print(len(data[(data['AE_0.01_Labels']==-1) | (data['IF_0.01_Labels']==-1) ]))
print(len(data[(data['AE_0.01_Labels']==-1) & (data['IF_0.01_Labels']==-1)]))

11897
8103


In [118]:
8103/11897

0.6810960746406657

In [119]:
print(len(data[(data['AE_0.01_Labels']==-1) & (data['EE_0.01_Lables']==-1)]))

4640


In [120]:
4640/11897

0.39001428931663445

In [121]:
data.to_csv('anomaly_all_alg_emsemble.csv')

In [124]:
# Intra cluster distance 
start = time.time()
dataNum['Outlier'] = auto_labels
benign = dataNum[dataNum['Outlier'] == 1]
anomalies = dataNum[dataNum['Outlier'] == -1]
benign_centroid = benign.mean().values
full_centroid = dataNum.mean().values

full_spread = 0
for index, row in dataNum.iterrows():
    d = np.sum(np.square(full_centroid - row.values))
    full_spread = full_spread + d
    
benign_spread = 0
for index, row in benign.iterrows():
    d = np.sum(np.square(benign_centroid - row.values))
    benign_spread = benign_spread + d
    
ratio = (full_spread / dataNum.shape[0]) / (benign_spread / benign.shape[0])
print(ratio)
end = time.time()
print("Time: ", end - start)

C:\Users\cathe\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


1374041.705420883
Time:  263.96994519233704
